# Tcell analysis in 3D

# 1. Assigning 3D info to 2D features

This code allows the detection of cell labels in 3D, by adding the information of which 3D cell label each section in the 2D analysis belongs to to the data set including the 2D features. 

In [22]:
from Cells import Cells
from MGFeatures import *
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import napari
from tifffile import imsave
import skimage.measure
import time
import seaborn as sns
import matplotlib.pyplot as plt
import cc3d
from progressbar import ProgressBar
from scipy import stats
import itertools

In [2]:
def get_area(labels, labels_list): 
    areas = []
    for i in labels_list:
        area = np.count_nonzero(labels == i)
        areas.append(area)
    areas = np.array(areas)
    return areas

In [3]:
folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\cell_final'
heterochromatin_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\heterochromatin_final'
gold_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\gold_final'
lysosomes_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\lysosomes_final'
mitochondria_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\mitochondria_final'
nucleus_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\nucleus_final'
ER_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\er_final'
golgi_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\golgi_final'

#### Create cell labels in 3d
1. Create a matrix with the size of the images (in this case 37440x35168) and the depth of the total number of images in z that we want to compare at the same time. The total number of images that can be compared depends on CPU and RAM capacity. 

2. The for loop will add binary images of each section to cell_stack.

3. cc3d.connected_components() will assign 3d labels based on the binary information in the cell_stack and store it in the matrix cell_labels. The connectivity value can be set to 6, 18 or 26. For more information (https://pypi.org/project/connected-components-3d/).

In [4]:
cell_stack = np.zeros(shape = (5, 37440, 35168), dtype = int8)
for i in range(0,5):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    binary_image = cell_image.astype(bool)
    cell_stack[i] = binary_image    #*NOTE*: if running from a range that does not start from 0, adjust the index
cell_labels = cc3d.connected_components(cell_stack, connectivity=18)

#### function features_to_3d
This function uses the cell_labels matrix and the features dataframe, which contains the 2D features calculated in the 2D analysis. 
- cell_labels[j].max() calculates per section which is the maximum number of cell labels in that section.
- np.where(cell_labels[j] == i), calculates in which coordinates of x and y in a certain layer j, it corresponds to the cell label i.
- The rest of the script compares the location in x and y of the cell labels stored in 'layer' with the coordinates in which each cell label in 2d is located. If one cell section is located inside of a cell label in 3d, it gets assigned its identificator value. 
- In case the range does not start in 0, add the number of the starting section in x.
- Include start section as j and final section as z.

In [20]:
def features_to_3d(cell_labels, features, j, z, x):
    featuresnew =  features.copy()
    featuresnew['id_3d'] = 0
    pbar = ProgressBar()
    for j in pbar(range(j,z)):
        for i in range(1, cell_labels[j].max()+1):
            layer = np.where(cell_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(featuresnew)):
                    if featuresnew.loc[n, 'sl_num'] == j + x:
                        if (layer[0].min() >= featuresnew['x1'][n]) and (layer[0].max() <= featuresnew['x2'][n]) and (layer[1].min() >= featuresnew['y1'][n]) and (layer[1].max() <= featuresnew['y2'][n]):
                            featuresnew['id_3d'][n] = i
    return featuresnew

In [13]:
features2d = pd.read_excel('total_features_v5.xlsx', index_col=0)

Filter the 2d features file to the number of sections to be run with features_to_3d

In [14]:
features5 = pd.DataFrame()
features5 = features2d[(features2d['sl_num'] >= 0) & (features2d['sl_num']  < 5)].reset_index()

In [21]:
start_time = time.time()
features5_3d = features_to_3d(cell_labels, features5, 0, 5, 0)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

  0% (0 of 5) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--C:\Users\Gast-User\AppData\Local\Temp\ipykernel_28084\1584851351.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  featuresnew['id_3d'][n] = i
100% (5 of 5) |##########################| Elapsed Time: 0:06:44 Time:  0:06:44


Elapsed time:  404.2144966125488


In [19]:
features5_3d 

# 2. 3D labelling for mitochondria

Same procedure as for assigning labels to each cell but in this cae for mitochondria. 
This also includes the area of each individual mitochondria and its coordinates, beside creating mito_stack for caluclating later on the connected components in 3d.
In the same way as in the cell labels, run this for as many sections as possible.

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_mito_data = []
mito_stack = np.zeros(shape = (5, 37440, 35168), dtype = int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
mito_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(0, 5)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    mitochondria_labels = mask_it(cell_labels, cell.img_from_tiles(mitochondria_folder))
    mito_label = skimage.measure.label(mitochondria_labels, connectivity=2)
    mito_coord = []
    region = regionprops(mito_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        mito_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_mito_data.extend(mito_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    mito_list = np.unique(mito_label)[1:]
    mito_area_values = get_area(mito_label, mito_list)
    mito_area_values_list.append(mito_area_values)
    mito_stack[i] = mito_label
# Create the coord_mito DataFrame from the collected data
coord_mito = pd.DataFrame(all_mito_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
mito_area_values_combined = np.concatenate(mito_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_mito['mito_area'] = mito_area_values_combined
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

Calculation of the maximum value of the side of each cell and the volume.
Axial resolution: 200nm and lateral resolution 10nm -> height is 20 times a pixel in the lateral.
## IMPORTANT NOTE
If the old excel files are used, these were run calculating the volume by multiplying the area * 200. So if these files are used, the volume has to be divided by 10.
However, for running new files, use area * 20.

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_mito['mito_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in range(len(coord_mito)):
    D = max(coord_mito['max_row'][y] - coord_mito['min_row'][y], coord_mito['max_col'][y] - coord_mito['min_col'][y])
    coord_mito.loc[y, 'mito_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)
coord_mito['volumen'] = coord_mito['mito_area'] * 20

In [ ]:
mito_labels = cc3d.connected_components(mito_stack, connectivity=26)

#### Function mito_to_3d
This function performs the same as features_to_3d, but takes into account the difference in the names of the columns 
(Perhaps this could be changed)
- In case the range does not start in 0, add the number of the starting section in x.
- Include start section as j and final section as z.

In [ ]:
def mito_to_3d(mito_labels, coord_mito, j, z, x):
    pbar = ProgressBar()
    features_mito =  coord_mito.copy()
    features_mito['mito_3d'] = 0
    for j in pbar(range(j, z)):
        for i in range(1, mito_labels[j].max()+1):
            layer = np.where(mito_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_mito)):
                    if features_mito.loc[n, 'sl_num'] == j + x:
                        if (layer[0].min() >= features_mito['min_row'][n]) and (layer[0].max() <= features_mito['max_row'][n]) and (layer[1].min() >= features_mito['min_col'][n]) and (layer[1].max() <= features_mito['max_col'][n]):
                            features_mito['mito_3d'][n] = i
    return features_mito

In [ ]:
start_time = time.time()
features_mito_5_3d = mito_to_3d(mito_labels, coord_mito, 0, 5, 0)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_mito_5_3d.to_excel('features_mito_5_3d_v1.xlsx')

# 3. 3D labelling for nucleus
Same as in mitochondria analysis

In [ ]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_nuc_data = []
nuc_stack_20 = np.zeros(shape = (10, 37440, 35168), dtype = np.int8)
# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
nuc_area_values_list = []
pbar = ProgressBar()
for i in pbar(range(10,20)):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    nuc_coord = []
    region = regionprops(nuc_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_nuc_data.extend(nuc_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    nuc_list = np.unique(nuc_label)[1:]
    nuc_area_values = get_area(nuc_label, nuc_list)
    nuc_area_values_list.append(nuc_area_values)
    nuc_stack_20[i-10] = nuc_label
# Create the coord_mito DataFrame from the collected data
coord_nuc_20 = pd.DataFrame(all_nuc_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
# Combine 'mito_area' values into a single list
nuc_area_values_combined = np.concatenate(nuc_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_nuc_20['nuc_area'] = nuc_area_values_combined

In [ ]:
start_time = time.time()
# Initialize the 'mito_large' column with zeros
coord_nuc_20['nuc_large'] = 0
pbar = ProgressBar()
# Calculate 'mito_large' values
for y in pbar(range(len(coord_nuc_20))):
    D = max(coord_nuc_20['max_row'][y] - coord_nuc_20['min_row'][y], coord_nuc_20['max_col'][y] -coord_nuc_20['min_col'][y])
    coord_nuc_20.loc[y, 'nuc_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)
coord_nuc_20['volumen'] = coord_nuc_20['nuc_area'] * 20

In [ ]:
nuc_labels_20 = cc3d.connected_components(nuc_stack_20, connectivity=26)

In [ ]:
def nuc_to_3d(nuc_labels, coord_nuc, j, z, x):
    pbar = ProgressBar()
    features_nuc =  coord_nuc.copy()
    features_nuc['nuc_3d'] = 0
    for j in pbar(range(j, z)):
        for i in range(1, nuc_labels[j].max()+1):
            layer = np.where(nuc_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_nuc)):
                    if features_nuc.loc[n, 'sl_num'] == j + x:
                        if (layer[0].min() >= features_nuc['min_row'][n]) and (layer[0].max() <= features_nuc['max_row'][n]) and (layer[1].min() >= features_nuc['min_col'][n]) and (layer[1].max() <= features_nuc['max_col'][n]):
                            features_nuc['nuc_3d'][n] = i
    return features_nuc

In [ ]:
start_time = time.time()
features_nuc_20_3d = nuc_to_3d(nuc_labels_20, coord_nuc_20, 10, 20, 10)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [ ]:
features_nuc_20_3d.to_excel('features_nuc_20_3d_v1.xlsx')

# 4. Combine all individual features files into one dataframe for cell, mitochondria and nucleus 

## 4.1 Cell features

##### sphericity
To measure the 3D morphology of the cell we used the sphericity, which is defined as:


$$sphericity = \sqrt[3]{\frac{Volume}{Volume_{circ}}}$$

In which Volume is the real volume of the particle and Volumecirc is the volume of the sphere around it

Wadell, H. (1934) “Shape determination of large sedimental rock fragments”. The PanAmerican Geologist. Vol. 61, pp. 187-220.

In [ ]:
features3d = pd.DataFrame()
# Load all files with 3d cell features
features3d_78 = pd.read_excel(r'C:\Users\Gast-User\Downloads\features78_v2.xlsx', index_col=0)
features3d_155 = pd.read_excel(r'C:\Users\Gast-User\Downloads\features155_v2.xlsx', index_col=0)
# Add the max label in one file to the next, so that labels from different cells do not have an identical value
features3d_155_max = features3d_155.copy()
features3d_155_max['id_3d'] = features3d_155_max['nuc_3d'] + features3d_78['nuc_3d'].max()
features3d = features3d.append(features3d_78) 
features3d = features3d.append(features3d_155_max) 

In [ ]:
features3d.to_excel('features3d_all.xlsx')

Select only features in the data frame that will be later used in the analysis

In [ ]:
features3d_sel = features3d[['cell_area', 'sum_mito_area', 'nucleus_area', 'heterochromatin_area', 'lysosomal_area', 'gold_area',
                            'cytoplasm_area', 'ER_lengths', 'sl_num', 'x1', 'x2', 'y1', 'y2', 'id_3d']]

##### features3d_grouped
Contains the average per cell of features that describe the sections area of a certain organelle.

In [ ]:
features3d_grouped = features3d_sel.groupby(by='id_3d').mean().reset_index()
features3d_grouped = features3d_grouped[['id_3d', 'cell_area', 'sum_mito_area', 'nucleus_area', 'heterochromatin_area', 'lysosomal_area', 'gold_area',
                            'cytoplasm_area', 'ER_lengths']]

##### features3d_counts
Stores in 'counts' the number of sections that belong to the same 'id_3d' and the column 'height' indicates the height of the cell in the z axis.

In [ ]:
features3d_counts = features3d.groupby('id_3d', dropna=False).size().reset_index(name='counts')
features3d_counts['height'] = features3d_counts['counts']*20

##### features3d_large
The largest side from the bbox that surrounds each cell section in 2D is gonna be calculated and stored in the 'large_side' column. 
##### features3d_max_side
Once the largest side per each section in 2d is calculated, this groups the cells by the id_3d index and calculates the maximum side of all the cell in 3D. 

In [ ]:
features3d_large = features3d.copy()
features3d_large['large_side'] = 0
for y in range(len(features3d_large)):
    D = max(features3d_large['x2'][y] - features3d_large['x1'][y], features3d_large['y2'][y] - features3d_large['y1'][y])
    features3d_large['large_side'][y] = D
features3d_max_side = features3d_large.groupby('id_3d').max().reset_index()
features3d_max_side = features3d_max_side[['id_3d', 'large_side']]
features3d_max_side

##### features3d_diameter
This data frame allows knowing the diameter of the Volume circ, which is the minimum sphere that encapsulates the cell 3d. The diameter of this sphere is then either going to be the largest side of the cell in the lateral axis (the maximum side of the bboxes of each section) or in the axial axis (height of the 3d object).
Then, the formula to calculate the volume of an sphere is applied.

In [ ]:
features3d_diameter = pd.merge(features3d_counts, features3d_max_side, on='id_3d')
features3d_diameter['diameter'] = 0
for i in range(len(features3d_diameter)):
    if features3d_diameter['large_side'][i] >= features3d_diameter['height'][i]:
        features3d_diameter['diameter'][i] = features3d_diameter['large_side'][i]
    else:
        features3d_diameter['diameter'][i] = features3d_diameter['height'][i]
diameter = features3d_diameter['diameter']
radius = diameter / 2
volume = (4/3) * np.pi * (radius ** 3)
features3d_diameter['volume_circ'] = volume

##### features3d_volume
This calculates the total volume of the cell, which is gonna be the sum of all section volumes, which are calculated by their area and the height (200nm).

In [ ]:
features3d_volume = features3d[['cell_area', 'id_3d']]
features3d_volume['volume']= features3d['cell_area'] * 20
features3d_volume = features3d_volume.groupby('id_3d').sum().reset_index()
features3d_volume = features3d_volume[['id_3d', 'volume']]

In [ ]:
features3d_sphere = pd.merge(features3d_diameter, features3d_volume, on = 'id_3d')
features3d_sphere['sphericity'] = (features3d_sphere['volume']/features3d_sphere['volume_circ'])**(1./3.)
features3d_sphere

##### features3d_coord
Calculates the min and max values of the total cell. This is used later on to plot the cell in napari.

In [ ]:
features3d_min = features3d.groupby('id_3d').min().reset_index()
features3d_min = features3d_min[['id_3d', 'x1', 'y1']]
features3d_max = features3d.groupby('id_3d').max().reset_index()
features3d_max = features3d_max[['id_3d', 'x2', 'y2']]
features3d_coord = pd.merge(features3d_min, features3d_max, on= 'id_3d')
features3d_coord

##### features3d_total
We merge the different data frames.

In [ ]:
features3d_merged = pd.merge(features3d_sphere, features3d_grouped, on='id_3d')
features3d_total = pd.merge(features3d_merged, features3d_coord, on='id_3d')
features3d_total = features3d_total.rename(columns = {'id_3d':'cell_3d'})
features3d_total.to_excel('features3d_total_v2.xlsx')

## 4.2 Mitochondria features
Same calculations in different data frames and then their combination is used for both the mitochondria and the nuclues.

In [ ]:
features_mito_30 = pd.read_excel(r'C:\Users\Gast-User\Desktop\MGFeatures-main\features_mito_30_3d_v1.xlsx', index_col=0)
features_mito_60 = pd.read_excel(r'C:\Users\Gast-User\Desktop\MGFeatures-main\features_mito_60_3d_v1.xlsx', index_col=0)
features_mito_90 = pd.read_excel(r'C:\Users\Gast-User\Desktop\MGFeatures-main\features_mito_90_3d_v1.xlsx', index_col=0)
features_mito_120 = pd.read_excel(r'C:\Users\Gast-User\Desktop\MGFeatures-main\features_mito_120_3d_v1.xlsx', index_col=0)
features_mito_155 = pd.read_excel(r'C:\Users\Gast-User\Desktop\MGFeatures-main\features_mito_155_3d_v1.xlsx', index_col=0)

In [ ]:
features_mito_60_max = features_mito_60.copy()
features_mito_60_max['mito_3d'] = features_mito_60_max['mito_3d'] + features_mito_30['mito_3d'].max()
features_mito_90_max = features_mito_90.copy()
features_mito_90_max['mito_3d'] = features_mito_90_max['mito_3d'] + features_mito_60_max['mito_3d'].max()
features_mito_120_max = features_mito_120.copy()
features_mito_120_max['mito_3d'] = features_mito_120_max['mito_3d'] + features_mito_90_max['mito_3d'].max()
features_mito_155_max = features_mito_155.copy()
features_mito_155_max['mito_3d'] = features_mito_155_max['mito_3d'] + features_mito_120_max['mito_3d'].max()

In [ ]:
features_mito = pd.DataFrame()
features_mito = features_mito.append(features_mito_30)
features_mito = features_mito.append(features_mito_60_max)
features_mito = features_mito.append(features_mito_90_max)
features_mito = features_mito.append(features_mito_120_max)
features_mito = features_mito.append(features_mito_155_max)
features_mito

In [ ]:
features_mito.reset_index(drop=True, inplace=True)

We calculate which cell 3d label belongs to each mito 3d label based on their coordinates.

In [ ]:
features_mito['cell_3d'] = 0
pbar = ProgressBar()
for i in pbar(range(len(features_mito))):
    for j in range(len(features_3d)):
        if (features_mito['min_row'][i] > features_3d['x1'][j]) and (features_mito['min_col'][i] > features_3d['y1'][j]) and (features_mito['max_row'][i] < features_3d['x2'][j]) and (features_mito['max_col'][i] < features_3d['y2'][j]):
            features_mito['cell_3d'][i] = features_3d['id_3d'][j]

In [ ]:
features_mito.to_excel('features_mito_all_v3.xlsx')

##### features_mito_sum_volume
'mito_sum_volume' represents the total volume that all mitochondria occupy in a cell. 
##### features_mito_area
Calculates per mitochondria the average area.
##### features_mito_volume
Calculates the volume per mitochondria

In [ ]:
features_mito_sum_volume = features_mito.groupby('cell_3d').sum().reset_index()
features_mito_sum_volume = features_mito_sum_volume[['cell_3d','volumen']]
features_mito_sum_volume.rename(columns = {'volumen':'mito_sum_volume'}, inplace = True)

In [ ]:
features_mito_area = features_mito.groupby('mito_3d').mean().reset_index()
features_mito_area = features_mito_area[['mito_area', 'mito_3d']]

In [ ]:
features_mito_volume = features_mito.groupby('mito_3d').sum().reset_index()
features_mito_volume.reset_index(drop=True)
features_mito_volume = features_mito_volume[['volumen', 'mito_3d']]

##### Mitochondria sphericity

In [ ]:
features_mito_counts = features_mito.groupby('mito_3d', dropna=False).size().reset_index(name='counts')
features_mito_counts['height'] = 0
features_mito_counts['height'] = features_mito_counts['counts']*20

In [ ]:
features_mito_large = pd.DataFrame()
features_mito_large = features_mito.groupby('mito_3d').max().reset_index()
features_mito_large = features_mito_large[['mito_large', 'mito_3d']]

In [ ]:
features_mito_diameter = pd.merge(features_mito_counts, features_mito_large, on='mito_3d')
features_mito_diameter['diameter'] = 0
for i in range(len(features_mito_diameter)):
    if features_mito_diameter['mito_large'][i] >= features_mito_diameter['height'][i]:
        features_mito_diameter['diameter'][i] = features_mito_diameter['mito_large'][i]
    else:
        features_mito_diameter['diameter'][i] = features_mito_diameter['height'][i]

In [ ]:
diameter = features_mito_diameter['diameter']
radius = diameter / 2
volume = (4/3) * np.pi * (radius ** 3)
features_mito_diameter['volume_circ'] = volume

In [ ]:
features_mito_sphericity = pd.merge(features_mito_diameter, features_mito_volume, on= 'mito_3d')
features_mito_sphericity['sphericity'] = (features_mito_sphericity['volumen']/features_mito_sphericity['volume_circ'])**(1./3.)

#### Merging
All previous features are calculated per mitochondria, so now the mean value of sphericity, mito_area and counts of all mitochondria in one cell is calculated. 
Also, the sum_mito_volume that already includes the total mitochondria volume in one cell is added.

In [ ]:
features_mito_3d = features_mito[['mito_3d', 'cell_3d']]
features_mito_final = pd.DataFrame()
features_mito_final = pd.merge(features_mito_sphericity, features_mito_3d, on='mito_3d')
features_mito_final = features_mito_final_mean.groupby('cell_3d').mean().reset_index()
features_mito_final = features_mito_final[['cell_3d', 'counts', 'mito_area', 'sphericity']]
features_mito_final_volume = pd.merge(features_mito_final_mean, features_mito_sum_volume, on='cell_3d')
features_mito_final_volume = features_mito_final_volume.rename(columns = {'counts': 'mito_number', 'sphericity': 'mito_sphericity'})
features_mito_final_volume.to_excel('features_mito_final_volume_v2.xlsx')

## 4.3 Nuclear features
### NOTE
In the old files, features_nuc_20, features_nuc_30 and features_nuc_40, are missing the nuc_large values, so that has to be calculated if using the raw files instead of the final file that contains all sections

In [ ]:
features_nuc_10 = pd.read_excel('features_nuc_10_3d_v1.xlsx', index_col=0)
features_nuc_20 = pd.read_excel('features_nuc_20_3d_v1.xlsx', index_col=0)
features_nuc_30 = pd.read_excel('features_nuc_30_3d_v1.xlsx', index_col=0)
features_nuc_40 = pd.read_excel('features_nuc_40_3d_v1.xlsx', index_col=0)
features_nuc_50 = pd.read_excel('features_nuc_50_3d_v1.xlsx', index_col=0)
features_nuc_60 = pd.read_excel('features_nuc_60_3d_v1.xlsx', index_col=0)
features_nuc_70 = pd.read_excel('features_nuc_70_3d_v1.xlsx', index_col=0)
features_nuc_80 = pd.read_excel('features_nuc_80_3d_v1.xlsx', index_col=0)
features_nuc_90 = pd.read_excel('features_nuc_90_3d_v1.xlsx', index_col=0)
features_nuc_100 = pd.read_excel('features_nuc_100_3d_v1.xlsx', index_col=0)
features_nuc_110 = pd.read_excel('features_nuc_110_3d_v1.xlsx', index_col=0)
features_nuc_120 = pd.read_excel('features_nuc_120_3d_v1.xlsx', index_col=0)
features_nuc_130 = pd.read_excel('features_nuc_130_3d_v1.xlsx', index_col=0)
features_nuc_140 = pd.read_excel('features_nuc_140_3d_v1.xlsx', index_col=0)
features_nuc_155 = pd.read_excel('features_nuc_155_3d_v1.xlsx', index_col=0)

In [ ]:
features_nuc_20_max = features_nuc_20.copy()
features_nuc_20_max['nuc_3d'] = features_nuc_20_max['nuc_3d'] + features_nuc_10['nuc_3d'].max()
features_nuc_30_max = features_nuc_30.copy()
features_nuc_30_max['nuc_3d'] = features_nuc_30_max['nuc_3d'] + features_nuc_20['nuc_3d'].max()
features_nuc_40_max = features_nuc_40.copy()
features_nuc_40_max['nuc_3d'] = features_nuc_40_max['nuc_3d'] + features_nuc_30['nuc_3d'].max()
features_nuc_50_max = features_nuc_50.copy()
features_nuc_50_max['nuc_3d'] = features_nuc_50_max['nuc_3d'] + features_nuc_40['nuc_3d'].max()
features_nuc_60_max = features_nuc_60.copy()
features_nuc_60_max['nuc_3d'] = features_nuc_60_max['nuc_3d'] + features_nuc_50['nuc_3d'].max()
features_nuc_70_max = features_nuc_70.copy()
features_nuc_70_max['nuc_3d'] = features_nuc_70_max['nuc_3d'] + features_nuc_60['nuc_3d'].max()
features_nuc_80_max = features_nuc_80.copy()
features_nuc_80_max['nuc_3d'] = features_nuc_80_max['nuc_3d'] + features_nuc_70['nuc_3d'].max()
features_nuc_90_max = features_nuc_90.copy()
features_nuc_90_max['nuc_3d'] = features_nuc_90_max['nuc_3d'] + features_nuc_80['nuc_3d'].max()
features_nuc_100_max = features_nuc_100.copy()
features_nuc_100_max['nuc_3d'] = features_nuc_100_max['nuc_3d'] + features_nuc_90['nuc_3d'].max()
features_nuc_110_max = features_nuc_110.copy()
features_nuc_110_max['nuc_3d'] = features_nuc_110_max['nuc_3d'] + features_nuc_100['nuc_3d'].max()
features_nuc_120_max = features_nuc_120.copy()
features_nuc_120_max['nuc_3d'] = features_nuc_120_max['nuc_3d'] + features_nuc_110['nuc_3d'].max()
features_nuc_130_max = features_nuc_130.copy()
features_nuc_130_max['nuc_3d'] = features_nuc_130_max['nuc_3d'] + features_nuc_120['nuc_3d'].max()
features_nuc_140_max = features_nuc_140.copy()
features_nuc_140_max['nuc_3d'] = features_nuc_140_max['nuc_3d'] + features_nuc_130['nuc_3d'].max()
features_nuc_155_max = features_nuc_155.copy()
features_nuc_155_max['nuc_3d'] = features_nuc_155_max['nuc_3d'] + features_nuc_140['nuc_3d'].max()

In [ ]:
features_nuc = pd.DataFrame()
features_nuc = features_nuc.append(features_nuc_10)
features_nuc = features_nuc.append(features_nuc_20_max)
features_nuc = features_nuc.append(features_nuc_30_max)
features_nuc = features_nuc.append(features_nuc_40_max)
features_nuc = features_nuc.append(features_nuc_50_max)
features_nuc = features_nuc.append(features_nuc_60_max)
features_nuc = features_nuc.append(features_nuc_70_max)
features_nuc = features_nuc.append(features_nuc_80_max)
features_nuc = features_nuc.append(features_nuc_90_max)
features_nuc = features_nuc.append(features_nuc_100_max)
features_nuc = features_nuc.append(features_nuc_110_max)
features_nuc = features_nuc.append(features_nuc_120_max)
features_nuc = features_nuc.append(features_nuc_130_max)
features_nuc = features_nuc.append(features_nuc_140_max)
features_nuc = features_nuc.append(features_nuc_155_max)

In [ ]:
features_nuc['cell_3d'] = 0
pbar = ProgressBar()
for i in pbar(range(len(features_nuc))):
    for j in range(len(features_3d)):
        if (features_nuc['min_row'][i] > features_3d['x1'][j]) and (features_nuc['min_col'][i] > features_3d['y1'][j]) and (features_nuc['max_row'][i] < features_3d['x2'][j]) and (features_nuc['max_col'][i] < features_3d['y2'][j]):
            features_nuc['cell_3d'][i] = features_3d['id_3d'][j]

In [ ]:
features_nuc.to_excel('features_nuc_all_v3.xlsx')

In [ ]:
features_nuc_sum_volume = features_nuc.groupby('cell_3d').sum().reset_index()
features_nuc_sum_volume = features_nuc_sum_volume[['cell_3d','volumen']]
features_nuc_sum_volume.rename(columns = {'volumen':'nuc_sum_volume'}, inplace=True)

In [ ]:
features_nuc_area = features_nuc.groupby('nuc_3d').mean().reset_index()
features_nuc_area = features_nuc_area[['nuc_area', 'nuc_3d']]

In [ ]:
features_nuc_volume = features_nuc.groupby('nuc_3d').sum().reset_index()
features_nuc_volume.reset_index(drop=True)
features_nuc_volume = features_nuc_volume[['volumen', 'nuc_3d']]

In [23]:
features_nuc_counts = features_nuc.groupby('nuc_3d', dropna=False).size().reset_index(name='counts')
features_nuc_counts['height'] = 0
features_nuc_counts['height'] = features_nuc_counts['counts']*100

NameError: name 'features_nuc' is not defined

In [ ]:
features_nuc_large = pd.DataFrame()
features_nuc_large = features_nuc.groupby('nuc_3d').max().reset_index()
features_nuc_large = features_nuc_large[['nuc_large', 'nuc_3d']]

features_nuc_diameter = pd.merge(features_nuc_counts, features_nuc_large, on='nuc_3d')
features_nuc_diameter['diameter'] = 0
for i in range(len(features_nuc_diameter)):
    if features_nuc_diameter['nuc_large'][i] >= features_nuc_diameter['height'][i]:
        features_nuc_diameter['diameter'][i] = features_nuc_diameter['nuc_large'][i]
    else:
        features_nuc_diameter['diameter'][i] = features_nuc_diameter['height'][i]
diameter = features_nuc_diameter['diameter']
radius = diameter / 2
volume = (4/3) * np.pi * (radius ** 3)
features_nuc_diameter['volume_circ'] = volume
features_nuc_diameter

In [ ]:
features_nuc_sphericity = pd.merge(features_nuc_diameter, features_nuc_volume, on= 'nuc_3d')
features_nuc_sphericity['sphericity'] = (features_nuc_sphericity['volumen']/features_nuc_sphericity['volume_circ'])**(1./3.)
features_nuc_sphericity = pd.merge(features_nuc_sphericity, features_nuc_area, on='nuc_3d')

In [ ]:
features_nuc_3d = features_nuc[['nuc_3d', 'cell_3d']]
features_nuc_final = pd.DataFrame()
features_nuc_final = pd.merge(features_nuc_sphericity, features_nuc_3d, on='nuc_3d')
features_nuc_final= features_nuc_final.groupby('cell_3d').mean().reset_index()

In [ ]:
features_nuc_final_volume = pd.merge(features_nuc_final, features_nuc_sum_volume, on='cell_3d')
features_nuc_final_volume = features_nuc_final_volume.rename(columns = {'counts':'nuc_number', 'sphericity':'nuc_sphericity'})
features_nuc_final_volume = features_nuc_final_volume[['cell_3d', 'nuc_number', 'nuc_sphericity', 'nuc_sum_volume', 'nuc_area']]
features_nuc_final_volume.to_excel('features_nuc_final_volume_v3.xlsx')

## 4.4 Combine cell, mito and nuc features in 3d

In [ ]:
features3d_total = pd.read_excel('features3d_total_v2.xlsx', index_col = 0)
features_mito_total = pd.read_excel('features_mito_final_volume_v2.xlsx', index_col = 0)
features_nuc_total = pd.read_excel('features_nuc_final_volume_v3.xlsx', index_col = 0)

In [ ]:
features3d_mito_total = pd.merge(features3d_total, features_mito_final_volume, on = 'cell_3d')
features3d_mito_nuc = pd.merge(features3d_mito_total, features_nuc_final_volume, on = 'cell_3d')
features3d_mito_nuc.to_excel('features3d_mito_nuc_v4.xlsx')

#### Selection
We select only cells that have at least a count of 5, meaning they have at least 5 sections; and less than 50, as cell_3d 0 has a huge number of cells.

In [ ]:
features3d_mito_nuc_sel = features3d_mito_nuc[(features3d_mito_nuc['counts'] >= 5) & (features3d_mito_nuc['counts'] < 50)]

In [ ]:
features3d_mito_nuc_sel.to_excel('features3d_mito_nuc_sel_v4.xlsx')

# 5. PCA analysis and kmeans clustering of 3D features
Explanation of the code and the steps are equal to those in 2d_tcell_analysis.ipynb

In [ ]:
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
featuresraw = pd.read_excel('features3d_mito_nuc_sel_v4.xlsx',  index_col=0)

#### ratios_3d
Additional features can be added to this data frame, such as the ratio between the sum_mito_volume/volume and the sum_nuc_volume/volume

In [ ]:
ratios_3d = pd.DataFrame()
ratios_3d['sphericity'] = featuresraw['sphericity']
ratios_3d['mito_sphericity'] = featuresraw['mito_sphericity']
ratios_3d['nuc_sphericity'] = featuresraw['nuc_sphericity']
ratios_3d['mito_cytoplasm_R'] = featuresraw['mito_area']/featuresraw['cytoplasm_area']
ratios_3d['nuc_cell_R'] = featuresraw['nuc_area']/featuresraw['cell_area']
ratios_3d['heterochromatin_nucleus_R'] = featuresraw['heterochromatin_area']/featuresraw['nucleus_area']
ratios_3d['gold_cytoplasm_R'] = featuresraw['gold_area']/featuresraw['cytoplasm_area']
ratios_3d['lysosome_cytoplasm_R'] = featuresraw['lysosomal_area']/featuresraw['cytoplasm_area']
ratios_3d['ER_lengths_R'] = featuresraw['ER_lengths']/featuresraw['cytoplasm_area']

In [ ]:
ratios_3d.fillna(0, inplace=True)

In [ ]:
scaler = StandardScaler() #Standardize features by removing the mean and scaling to unit variance.
std = scaler.fit_transform(ratios_3d) #returns a transformed array
pca = PCA()
pca.fit(std)

In [ ]:
pca.explained_variance_ratio_.cumsum()

In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(range(0,9), pca.explained_variance_ratio_.cumsum()) #use number of atributes for the range of the plot
plt.xlabel('Num component')
plt.ylabel('Cumul explained var')
plt.tight_layout()
plt.show()
#plt.savefig('acumulative_pca_mean.png')

In [ ]:
pca = PCA(n_components=3)
pca.fit(std)
scores_pca = pca.transform(std)

In [ ]:
wcss = []
for i in range(1,22):
    kmeans_pca = KMeans(n_clusters=i, init='k-means++', random_state=42) 
    kmeans_pca.fit(scores_pca)
    wcss.append(kmeans_pca.inertia_)

In [ ]:
plt.figure(figsize = (5,4))
plt.plot(range(1, 22), wcss, marker='o')
plt.xlabel('num clusters')
plt.ylabel('WCSS')
plt.tight_layout()
plt.show()
#plt.savefig('wcss_pca3.png')

In [ ]:
kmeans_pca = KMeans(n_clusters=4, init='k-means++', random_state=42)
kmeans_pca.fit(scores_pca)

In [ ]:
ratios_pca = pd.concat([ratios_3d.reset_index(drop = True), pd.DataFrame(scores_pca)], axis=1)
ratios_pca.columns.values[-3:] = ['Component1', 'Component2', 'Component3']
ratios_pca['pca_kmeans_labels'] = kmeans_pca.labels_

In [ ]:
ratios_plot = ratios_pca[['sphericity',
                         'mito_sphericity',
                        'nuc_sphericity',
                          'mito_cytoplasm_R',
                          'nuc_cell_R',
                            #'mito_volume_R',
                          #'nuc_volume_R',
                          'heterochromatin_nucleus_R',
                          'gold_cytoplasm_R',
                          'lysosome_cytoplasm_R',
                          'ER_lengths_R',
                         'pca_kmeans_labels'
                            ]]
ratios_plot = ratios_plot.fillna(0, inplace = True)

In [ ]:
cmap = {0:'#3a60c2',
       1: '#bf4dab', #purple
       2:'#f9526d', 
       3:'#f38928',
       4:'#b8c20e'
       }

clabels = [cmap.get(k) for k in ratios_plot['pca_kmeans_labels']]

In [ ]:
x_axis = ratios_pca['Component2']
y_axis = ratios_pca['Component1']
plt.figure(figsize=(10,8))
sns.scatterplot(data=ratios_pca, x='Component2', y='Component1', hue='pca_kmeans_labels', palette=cmap)
sns.scatterplot(s = 100)

plt.title('clusters by pca')
plt.savefig('final_pca3_cluster4.png')

In [ ]:
sns.set(rc={'figure.figsize':(100,120)})
g = sns.PairGrid(ratios_plot,
                 hue='pca_kmeans_labels',
                 palette=cmap
                )

g.map_offdiag(sns.scatterplot, alpha=0.8)
g.map_diag(sns.histplot)
g.add_legend()
plt.savefig('features3d_mean_5pca_5clusters.png')

#### violin plot
All features used in the PCA analysis are plotted individually in violin plots divided by the cluster number, which allows comparisons of the different clusters 

In [ ]:
plt.clf()
features = ['sphericity', 'mito_sphericity', 'nuc_sphericity',  'mito_cytoplasm_R','nuc_cell_R', 'heterochromatin_nucleus_R', 'gold_cytoplasm_R', 'lysosome_cytoplasm_R', 'ER_lengths_R']

# Create a 3x3 grid of subplots
fig, axes = plt.subplots(3, 3, figsize=(12, 10))
fig.suptitle('Features in 3D per cluster')

# Loop through each feature and create a violinplot in the corresponding subplot
for i, feature in enumerate(features):
    row, col = i // 3, i % 3
    sns.violinplot(data=ratios_plot, x='pca_kmeans_labels', y=feature, ax=axes[row, col], 
                   palette = cmap, legend = False)
    axes[row, col].set_title(feature)
axes[0, 0].set_ylim(0, 1.2)
axes[0, 1].set_ylim(0, 1.2)
axes[0, 2].set_ylim(0, 1.2)
axes[1, 0].set_ylim(-0.05, 0.1)
axes[1, 1].set_ylim(-0.1, 1.1)
axes[1, 2].set_ylim(-0.2, 1.1)
axes[2,0].set_ylim(-0.05,0.2)
axes[2,1].set_ylim(-0.05, 0.2)
axes[2,2].set_ylim(-0.05, 0.2)
axes[0, 0].set_title("Whole cell spherictiy")
axes[0, 1].set_title("Mean mitochondria spherictiy")
axes[0, 2].set_title("Mean nuclear spherictiy")
axes[1, 0].set_title("Mean mitochondria/whole cytoplasm area")
axes[1, 1].set_title("Mean nucleus/whole cell area")
axes[1, 2].set_title("Heterochromatin/nucleus area")
axes[2, 0].set_title("Gold/cytoplasm area")
axes[2, 1].set_title("Lysosome/cytoplasm area")
axes[2, 2].set_title("ER length/cytoplasm area")
for j in range(3):
    axes[2, j].set_xlabel("Cluster")

# Remove x-axis for all other rows except the last one
for i in range(2):
    for j in range(3):
        axes[i, j].xaxis.set_visible(False)
# Adjust spacing between subplots
plt.tight_layout()

# Show the plots

plt.savefig('violin_final_pca3_3clusters.png')

# 6. Statistical analysis

1. For each feature, we first run a Shapiro-Wilk Test to evaluate normality of the distribution. 
- If p-value > 0.05 for all clusters, we can consider that data to be normally distributed.
2. If normal distributed, we run a Levene test to evaluate equal variance of the data.
- If p-value > 0.05, we can assume an equal variance in the data.
3. We evaluate if two groups have significantly different mean by:
- a. Student t-test: if normally distributed and equal variances.
- b.Welch´s t-test: if normally distributed but cannot assume equal variances.
- c. Mann-Whitney t-test: if non-normally distirbuted.
    
Examples of how to run each test are shown below. For each feature, run 1. and 2. and depending on the results run 3a, 3b or 3c.

#### Shapiro-Wilk test

In [ ]:
subgroups = {}
# Iterate through unique values in the 'pca_kmeans_values' column
for value in ratios_plot['pca_kmeans_labels'].unique():
    if value in [0,2,3]:
        # Create a subgroup based on the unique value
        subgroup = ratios_plot[ratios_plot['pca_kmeans_labels'] == value]

        # Extract the data_column values for this subgroup
        data = subgroup['mito_sphericity']

        # Perform the Shapiro-Wilk test for normality
        stat, p = stats.shapiro(data)

        # Print or store the results
        print(f"Subgroup {value}:")
        print(f"Shapiro-Wilk test statistic: {stat}, p-value: {p}")
        subgroups[value] = subgroup

#### Levene test

In [ ]:
subgroups = {}

# Create an empty list to store data from each subgroup
data_lists = []

# Iterate through unique values in the 'pca_kmeans_labels' column
for value in ratios_plot['pca_kmeans_labels'].unique():
    if value in [0,2,4]:
        # Create a subgroup based on the unique value
        subgroup = ratios_plot[ratios_plot['pca_kmeans_labels'] == value]

        # Extract the 'sphericity' values for this subgroup
        data = subgroup['mito_sphericity']

        # Append the data to the list
        data_lists.append(data)

        # Print or store the results
        print(f"Subgroup {value}:")
        print(f"Number of observations: {len(data)}")

# Perform Levene's test for homogeneity of variances
stat, p = stats.levene(*data_lists)

# Print the overall result
print("\nOverall Levene's Test Result:")
print(f"Levene's test statistic: {stat}, p-value: {p}")

#### Student t-test

In [ ]:
subgroups = {}

# Iterate through unique values in the 'pca_kmeans_labels' column
for value in ratios_plot['pca_kmeans_labels'].unique():
    if value in [0,2,3]:
        # Create a subgroup based on the unique value
        subgroup = ratios_plot[ratios_plot['pca_kmeans_labels'] == value]
        subgroups[value] = subgroup

# Perform unpaired Student's t-test for pairwise comparisons within subgroups
for group1, group2 in itertools.combinations(subgroups.keys(), 2):
    data1 = subgroups[group1]['mito_sphericity']  # Replace 'my_data' with your actual column name
    data2 = subgroups[group2]['mito_sphericity']  # Replace 'my_data' with your actual column name

    t_stat, p_value = stats.ttest_ind(data1, data2)
    print(f"Comparison between Subgroup {group1} and Subgroup {group2}:")
    print(f"Student's t-test statistic: {t_stat}")
    print(f"P-value: {p_value}")
    print()

#### Welch´s t-test

In [ ]:
subgroups = {}

# Iterate through unique values in the 'pca_kmeans_labels' column
for value in df['pca_kmeans_labels'].unique():
    # Create a subgroup based on the unique value
    subgroup = df[df['pca_kmeans_labels'] == value]
    subgroups[value] = subgroup

# Perform Welch's t-test for pairwise comparisons within subgroups
for group1, group2 in itertools.combinations(subgroups.keys(), 2):
    data1 = subgroups[group1]['my_data']  # Replace 'my_data' with your actual column name
    data2 = subgroups[group2]['my_data']  # Replace 'my_data' with your actual column name

    t_stat, p_value = stats.ttest_ind(data1, data2, equal_var=False)
    print(f"Comparison between Subgroup {group1} and Subgroup {group2}:")
    print(f"Welch's t-test statistic: {t_stat}")
    print(f"P-value: {p_value}")
    print()

#### Mann-Whitney U test

In [ ]:
subgroups = {}

# Iterate through unique values in the 'pca_kmeans_labels' column
for value in ratios_plot['pca_kmeans_labels'].unique():
    if value in [0,2,3]:
        # Create a subgroup based on the unique value
        subgroup = ratios_plot[ratios_plot['pca_kmeans_labels'] == value]
        subgroups[value] = subgroup

# Perform unpaired Student's t-test for pairwise comparisons within subgroups
for group1, group2 in itertools.combinations(subgroups.keys(), 2):
    data1 = subgroups[group1]['heterochromatin_nucleus_R']  # Replace 'my_data' with your actual column name
    data2 = subgroups[group2]['heterochromatin_nucleus_R']  # Replace 'my_data' with your actual column name

    stat, p_value = stats.mannwhitneyu(data1, data2, alternative='two-sided')
    print(f"Comparison between Subgroup {group1} and Subgroup {group2}:")
    print(f"Student's t-test statistic: {t_stat}")
    print(f"P-value: {p_value}")
    print()

# 7. Napari visualization

In [ ]:
from MGFeatures import remake_bbox, make_bbox_stack

The coordinates have to be readjusted due to the way of the export of the segmentation, which does not match the size of the image

In [ ]:
featuresraw_2 = featuresraw.copy()
for i in range(len(featuresraw_2)):
    if featuresraw_2['y1'][i] > 28080:
        featuresraw_2['y1'][i] =  featuresraw_2['y1'][i]-37440
    if featuresraw_2['y2'][i] > 28080:
        featuresraw_2['y2'][i] =  featuresraw_2['y2'][i]-37440
featuresraw_3 = featuresraw_2.copy()
for i in range(len(featuresraw_3)):
    featuresraw_3['x1'][i] =  featuresraw_3['x1'][i]+8792
    featuresraw_3['x2'][i] =  featuresraw_3['x2'][i]+8792
    featuresraw_3['y1'][i] =  featuresraw_3['y1'][i]+9360
    featuresraw_3['y2'][i] =  featuresraw_3['y2'][i]+9360

In [ ]:
features_napari = featuresraw_3[['x1', 'x2', 'y1', 'y2']]
ratios_napari = pd.concat([ratios_plot, features_napari], axis = 1)

In [ ]:
# These two cells were eliminated in this case because of some errors in the coordinates
ratios_napari_2 = ratios_napari.copy()
ratios_napari_2 = ratios_napari_2.drop([20,36])

In [ ]:
def create_bbox(row):
    bbox = (row['x1'], row['y1'], row['x2'], row['y2'])
    rect = np.array([[bbox[0], bbox[1]], [bbox[2], bbox[1]], [bbox[2], bbox[3]], [bbox[0], bbox[3]]])
    return rect
# Apply the function to create the 'bbox' column
ratios_napari_2['bbox_rect'] = ratios_napari_2.apply(create_bbox, axis=1)

In [ ]:
viewer = napari.Viewer()
# Add to the napari viewer the location of the cells and colored them based on the cluster number they belong to
viewer.add_shapes(data=ratios_napari_2['bbox_rect'], face_color = clabels)